In [1]:
import pandas as pd
import glob
from pathlib import Path
import censusdata
import re
import missingno as msno

In [2]:
### Input
DATA_DIR = Path("/Users/edward/Documents/race-covid/data")
FILTERED_DATA_DIR = DATA_DIR / 'filtered'
CLEAN_DATA_DIR = DATA_DIR / 'clean'

COOK_DATA_FILES = list(FILTERED_DATA_DIR.glob("*.parquet"))
OUTPUT_DATA_DIR = CLEAN_DATA_DIR / 'movement_data.parquet'

In [3]:
def read_and_clean_dates(data_dir):
    df = pd.read_parquet(data_dir)
    df['week_num'] = pd.to_datetime(df.date_range_start).apply(lambda x: x.strftime("%V"))
    df['perc_stay_home'] = round((df.completely_home_device_count / df.device_count) * 100,2)
    
    return df[['perc_stay_home', 'week_num', 'date_range_start', 
              'origin_census_block_group',
              'completely_home_device_count', 
              'device_count', 'candidate_device_count']]

In [4]:
df = pd.concat([read_and_clean_dates(dir) for dir in COOK_DATA_FILES])

In [5]:
df.groupby(['week_num']).min()

,perc_stay_home,date_range_start,origin_census_block_group,completely_home_device_count,device_count,candidate_device_count
week_num,,,,,,
10,0.75,2020-03-07T00:00:00-05:00,170310101001,1,5,15
11,0.80,2020-03-09T00:00:00-04:00,170310101001,1,6,15
12,1.64,2020-03-16T00:00:00-04:00,170310101001,1,5,16
13,0.75,2020-03-23T00:00:00-04:00,170310101001,1,5,15
14,0.85,2020-03-30T00:00:00-04:00,170310101001,1,5,13


In [6]:
grouped_df = df.groupby(['week_num', 'origin_census_block_group'], as_index=False).sum()
grouped_df.perc_stay_home = round((grouped_df.completely_home_device_count / grouped_df.device_count) * 100,2)
grouped_df.head()

,week_num,origin_census_block_group,perc_stay_home,completely_home_device_count,device_count,candidate_device_count
0,10,170310101001,11.76,4,34,88
1,10,170310101002,31.67,57,180,464
2,10,170310101003,33.33,48,144,364
3,10,170310102011,37.85,67,177,330
4,10,170310102012,27.27,93,341,796


In [7]:
pivot_df = grouped_df.pivot(index='origin_census_block_group', columns='week_num', values='perc_stay_home')
pivot_df

week_num,10,11,12,13,14
origin_census_block_group,,,,,
170310101001,11.76,19.85,20.56,31.71,44.30
170310101002,31.67,34.29,43.00,51.54,52.72
170310101003,33.33,28.70,42.42,43.88,42.90
170310102011,37.85,36.17,49.13,52.69,56.17
170310102012,27.27,29.28,43.03,50.00,47.33
...,...,...,...,...,...
170318439003,25.93,25.41,45.65,51.65,52.78
170318439004,22.86,31.03,30.80,34.54,37.00
170318439005,28.79,25.96,38.78,52.63,47.68


In [8]:
# Collect census data
cookbg = censusdata.download('acs5', 2019,
                             censusdata.censusgeo([('state', '17'), ('county', '031'), ('block group', '*')]),
                             ['B23025_003E', 'B23025_005E', 'B15003_001E', 'B15003_002E', 'B15003_003E',
                              'B15003_004E', 'B15003_005E', 'B15003_006E', 'B15003_007E', 'B15003_008E',
                              'B15003_009E', 'B15003_010E', 'B15003_011E', 'B15003_012E', 'B15003_013E',
                              'B15003_014E', 'B15003_015E', 'B15003_016E', 'B01003_001E', 'B02008_001E', 
                             ])
cookbg['percent_unemployed'] = cookbg.B23025_005E / cookbg.B23025_003E * 100
cookbg['percent_nohs'] = (cookbg.B15003_002E + cookbg.B15003_003E + cookbg.B15003_004E
                          + cookbg.B15003_005E + cookbg.B15003_006E + cookbg.B15003_007E + cookbg.B15003_008E
                          + cookbg.B15003_009E + cookbg.B15003_010E + cookbg.B15003_011E + cookbg.B15003_012E
                          + cookbg.B15003_013E + cookbg.B15003_014E +
                          cookbg.B15003_015E + cookbg.B15003_016E) / cookbg.B15003_001E * 100
cookbg['total_pop'] = (cookbg.B01003_001E)
cookbg['white'] = (cookbg.B02008_001E)
cookbg['white_perc'] = round(cookbg.white / cookbg.total_pop * 100,2)

cookbg = cookbg[['percent_unemployed', 'percent_nohs', 'white_perc']]
cookbg.describe()

,percent_unemployed,percent_nohs,white_perc
count,3985.000000,3986.000000,3986.000000
mean,8.119927,13.269368,56.848728
std,8.851660,12.095908,32.974905
min,0.000000,0.000000,0.000000
25%,2.172097,3.908710,27.757500
50%,5.287356,9.899990,65.910000
75%,11.103352,19.543624,85.470000
max,67.857143,100.000000,100.000000


In [9]:
# Notice the negative correlation between pecentage white, and unemployment, and high school attainment
cookbg.corr()

,percent_unemployed,percent_nohs,white_perc
percent_unemployed,1.000000,0.240351,-0.549366
percent_nohs,0.240351,1.000000,-0.304151
white_perc,-0.549366,-0.304151,1.000000


In [10]:
cookbg['cbg'] = list(cookbg.reset_index()['index'].astype(str).str.split(', ', expand=True)[4].str.replace(r"[a-zA-Z:> ]",''))
cookbg.rename(columns={'cbg': 'origin_census_block_group'}, inplace=True)
# cookbg.origin_census_block_group = cookbg.origin_census_block_group.astype(int)

<ipython-input-10-26774cb7a855>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  cookbg['cbg'] = list(cookbg.reset_index()['index'].astype(str).str.split(', ', expand=True)[4].str.replace(r"[a-zA-Z:> ]",''))


In [11]:
cookbg.origin_census_block_group = cookbg.origin_census_block_group.astype(float)
merged_df = pivot_df.reset_index().merge(cookbg, on="origin_census_block_group")
merged_df.shape

(3985, 9)

In [12]:
final_df = merged_df.dropna()
final_df.shape

(3980, 9)

In [13]:
final_df.to_parquet(OUTPUT_DATA_DIR)